<a href="https://colab.research.google.com/github/gned0/NLP_stock_prediction/blob/main/longformer_linear_classifier_v3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Predizione di indici di borsa tramite fincial news sentiment analysis

Progetto per tirocinio

Studente: Gian Luca Nediani

E-mail: gianluca.nediani@studio.unibo.it

## Introduzione

A partire da quanto mostrato nel paper [Deep Learning for Event-Driven Stock Prediction](https://www.ijcai.org/Proceedings/15/Papers/329.pdf), l'obiettivo è sviluppare una rete neurale in grado di predire l'andamento del mercato azionario tramite metodi di sentiment analysis: valutando le news di carattere finanziario di un dato giorno si vuole predire se il giorno dopo il valore di un certo indice di borsa aumenterà o diminuirà. Come nel paper, l'indice di riferimento utilizzato è _S&P500_, un indice rappresentativo delle performance delle 500 aziende più quotate nella borsa statunitense.

Per comprendere il significato semantico delle news e fare valutazioni sull'andamento del mercato, gli autori del paper rappresentano le news finanziarie come degli eventi. In questo esperimento invece, si farà ricorso a un'architettura Transformer, l'attuale stato dell'arte nel _natural language processing_. Grazie all'encoder di questa architettura, sarà possibile generare degli embedding in grado di rappresentare in maniera ricca il significato semantico dei titoli di notizie finanziarie. Questi embedding saranno poi l'input per una rete neurale di classificazione.

Nel paper originale per realizzare una predizione per un dato giorno vengono utilizzate news finanziarie dell'intero mese precedente, pur sottolineando che quelle con impatto maggiore sono le news del giorno precedente. In questo esperimento vengono utilizzate solo le news del giorno precedente.

In [ ]:
!pip install yfinance
!pip install transformers

  Preparing metadata (setup.py) ... - done
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=baab3e95e9964f1d0a6760cb110c0dbea15341a15fd18b07b7066735c07b2c24
  Stored in directory: /root/.cache/pip/wheels/34/ba/79/c0260c6f1a03f420ec7673eff9981778f293b9107974679e36
Successfully built multitasking


In [ ]:
import transformers
import torch
import pandas as pd
import yfinance as yf
import numpy as np
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup, LongformerTokenizer, LongformerModel
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import defaultdict

Path dei pesi del modello transformer preaddestrato.

In [ ]:
MODEL_PATH = 'allenai/longformer-large-4096'

Viene utilizzata la GPU fornita da Colab in quanto il calcolo degli embedding e l'addestramento della rete neurale tramite CPU sarebbero troppo lenti.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [ ]:
CUDA_LAUNCH_BLOCKING = "1"

## Il dataset

Il dataset utilizzato in questo esperimento è ottenuto a partire da due dataset di news finanziarie, entrambi utilizzati nel paper [Deep Learning for Event-Driven Stock Prediction](https://www.ijcai.org/Proceedings/15/Papers/329.pdf). Essi racchiudono rispettivamente 450341 news di natura finanziaria provenienti dalla testata giornalistica _Bloomberg_ e 109110 news di natura finanziaria provenienti dalla testata giornalistica _Reuters_. Sulle orme del paper sopracitato, sono stati estratti soltanto i titoli delle news, in quanto considerati più significativi del corpo della notizia. Inoltre, siccome il modello sviluppato può processare un numero finito di informazioni, i titoli sono stati filtrati, mantenendo solo quelli che includano il nome di uno o più degli indici di borsa che compongono l'indice _S&P500_. 
Le operazioni preliminari appena descritte portano ad avere il seguente file CSV, che per ogni giorno del periodo preso in esame (2007-2016), unisce i titoli di Bloomberg e Reuters.

In [ ]:
import os.path
from urllib.request import urlretrieve

if not os.path.exists("financial_titles.csv"):
    urlretrieve("https://raw.githubusercontent.com/gned0/NLP_stock_prediction/main/all_financial_titles.csv", "financial_titles.csv")

df = pd.read_csv('financial_titles.csv', delimiter=',')
df = df.drop('Unnamed: 0', 1)
df = df.dropna(axis=0)
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,ts,title
0,20070102,Apple options probe spotlights ex-officials: p...
1,20070103,Ford CEO says restructuring going well. Ford s...
2,20070104,"US STOCKS-Indexes end up as Intel lifts techs,..."
3,20070105,Nasdaq says no decisions made about LSE stake....
4,20070107,"CES-UPDATE 2-Sony, Microsoft hit game console ..."
...,...,...
3059,20110305,AT&T Says John Stephens to Become CFO When Lin...
3060,20110312,Apple IPad 2 Lines Led by Gray Marketers Eager...
3061,20110414,Apple Is Said to Ready White IPhone Following ...
3062,20110917,"Samsung Seeks to Lift German Sales Ban, Sues A..."


In [ ]:
df['title'].astype(str).apply(lambda x: len(x.split())).mean()

763.6426240208878

In base al numero di parole medio delle entry del dataset, 1024 parole è la lunghezza massima scelta per gli input del modello. Gli input di lunghezza minore subiranno un padding, mentre quelli di lunghezza maggiore saranno tagliati.

In [ ]:
MAX_LEN = 1024

## Data preprocessing

Ottenuto il dataset dei titoli di news finanziarie, è necessario ottenere le informazioni relative all'andamento della borsa, in particolare dell'indice S&P500. Tramite il pacchetto yfinance viene creato un dataframe con informazioni sull'andamento di tale titolo (label ^GSPC) nel periodo corrispondente a quello coperto dal dataset di news.

In [ ]:
stock = yf.download("^GSPC", start="2007-01-01", end="2016-08-16")
stock

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-01-03,1418.030029,1429.420044,1407.859985,1416.599976,1416.599976,3429160000
2007-01-04,1416.599976,1421.839966,1408.430054,1418.339966,1418.339966,3004460000
2007-01-05,1418.339966,1418.339966,1405.750000,1409.709961,1409.709961,2919400000
2007-01-08,1409.260010,1414.979980,1403.969971,1412.839966,1412.839966,2763340000
2007-01-09,1412.839966,1415.609985,1405.420044,1412.109985,1412.109985,3038380000
...,...,...,...,...,...,...
2016-08-09,2182.239990,2187.659912,2178.610107,2181.739990,2181.739990,3334300000
2016-08-10,2182.810059,2183.409912,2172.000000,2175.489990,2175.489990,3254950000
2016-08-11,2177.969971,2188.449951,2177.969971,2185.790039,2185.790039,3423160000


L'obiettivo è ora di ottenere le etichette da usare per la classificazione delle giornate nel mercato azionario. Viene quindi creato un valore binario: 0 se in un dato giorno il valore dell'indice chiude in calo rispetto all'apertura e 1 se al contrario chiude in rialzo.

In [ ]:
def binarize(x):
  if x > 0:
    return 1
  return 0

In [ ]:
stock['target'] = 0
stock['target'] = stock['Close'] - stock['Open']
stock['target'] = stock['target'].apply(binarize)
stock.reset_index(inplace=True)
stock['Date'] = stock['Date'].astype(str).apply(lambda x: x.replace('-', ''))
stock.rename(columns={'Date':'ts'}, inplace = True)
stock


,ts,Open,High,Low,Close,Adj Close,Volume,target
0,20070103,1418.030029,1429.420044,1407.859985,1416.599976,1416.599976,3429160000,0
1,20070104,1416.599976,1421.839966,1408.430054,1418.339966,1418.339966,3004460000,1
2,20070105,1418.339966,1418.339966,1405.750000,1409.709961,1409.709961,2919400000,0
3,20070108,1409.260010,1414.979980,1403.969971,1412.839966,1412.839966,2763340000,1
4,20070109,1412.839966,1415.609985,1405.420044,1412.109985,1412.109985,3038380000,0
...,...,...,...,...,...,...,...,...
2417,20160809,2182.239990,2187.659912,2178.610107,2181.739990,2181.739990,3334300000,0
2418,20160810,2182.810059,2183.409912,2172.000000,2175.489990,2175.489990,3254950000,0
2419,20160811,2177.969971,2188.449951,2177.969971,2185.790039,2185.790039,3423160000,1
2420,20160812,2183.739990,2186.280029,2179.419922,2184.050049,2184.050049,3000660000,1


Le etichette vengono shiftate verso l'alto di una riga in quanto il problema in esame è di next day prediction: le news di un dato giorno influiscono sull'andamento dell'indice nel giorno successivo.

In [ ]:
stock['target'] = stock['target'].shift(-1)
stock.dropna(inplace=True)
stock = stock.astype('int64')
stock

,ts,Open,High,Low,Close,Adj Close,Volume,target
0,20070103,1418,1429,1407,1416,1416,3429160000,1
1,20070104,1416,1421,1408,1418,1418,3004460000,0
2,20070105,1418,1418,1405,1409,1409,2919400000,1
3,20070108,1409,1414,1403,1412,1412,2763340000,0
4,20070109,1412,1415,1405,1412,1412,3038380000,1
...,...,...,...,...,...,...,...,...
2416,20160808,2183,2185,2177,2180,2180,3327550000,0
2417,20160809,2182,2187,2178,2181,2181,3334300000,0
2418,20160810,2182,2183,2172,2175,2175,3254950000,1
2419,20160811,2177,2188,2177,2185,2185,3423160000,1


Viene ora fatto un merge fra il dataframe dei titoli e quello degli indici per ottenere il dataframe che verrà usato per addestramento e valutazione. Si noti che il numero finale di entry del dataframe sarà minore di quello iniziale dei titoli finanziari in quanto il mercato azionario non è aperto tutti i giorni, mentre le news nel dataset coprono tutti i 365 giorni dell'anno.

In [ ]:
df['ts'] = df['ts'].astype(str).astype(int)
stock = stock[['ts', 'target']]
df = df.merge(stock, on='ts')
df

,ts,title,target
0,20070103,Ford CEO says restructuring going well. Ford s...,1
1,20070104,"US STOCKS-Indexes end up as Intel lifts techs,...",0
2,20070105,Nasdaq says no decisions made about LSE stake....,1
3,20070108,Escala Group to be delisted from Nasdaq Jan 10...,0
4,20070109,"Chevron 4th-qtr liquid, natural gas production...",1
...,...,...,...
2366,20160809,Union drops joint employment claims against Mi...,0
2367,20160810,SolarCity says Tesla talks delayed closing pro...,1
2368,20160811,Peru detects fresh oil spill from decades-old ...,1
2369,20160812,Gilead to get attorney fees in hepatitis C pat...,1


In [ ]:
df = df[df['title'].astype(str).apply(lambda x: len(x.split())) > 128]
df

,ts,title,target
0,20070103,Ford CEO says restructuring going well. Ford s...,1
1,20070104,"US STOCKS-Indexes end up as Intel lifts techs,...",0
3,20070108,Escala Group to be delisted from Nasdaq Jan 10...,0
4,20070109,"Chevron 4th-qtr liquid, natural gas production...",1
5,20070110,eBay to buy ticketer StubHub for $310 million....,1
...,...,...,...
2366,20160809,Union drops joint employment claims against Mi...,0
2367,20160810,SolarCity says Tesla talks delayed closing pro...,1
2368,20160811,Peru detects fresh oil spill from decades-old ...,1
2369,20160812,Gilead to get attorney fees in hepatitis C pat...,1


In [ ]:
df['target'].value_counts()

1    1257
0    1056
Name: target, dtype: int64

A partire dal dataframe ottenuto viene creato un Dataset Pytorch e il corrispondente DataLoader sia per il set di addestramento che per quello di test. Il Dataset Pytorch contiene per ogni entry i token id delle parole e l'attention mask, necessaria in quanto in caso di entry più corte della massima lunghezza selezionata (1024 parole), si farà ricorso al padding. Il modello transformer che verrà utilizzato (longformer) richiede anche una global mask per indicare i token per i quali si vuole fare uso di attention globale. La maschera viene creata e impostata a 0 in quanto in questo caso non si farà uso di attention globale per nessun token.

In [ ]:
class FinancialDataset(Dataset):
  def __init__(self, titles, targets, tokenizer, max_len):
    self.titles = titles
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.titles)

  def __getitem__(self, item):
    title = str(self.titles[item])
    target = self.targets[item]
    input_ids = torch.tensor(self.tokenizer.encode(
      title, add_special_tokens=True, max_length=self.max_len, pad_to_max_length=True))
    attention_mask = torch.zeros(input_ids.shape, dtype=torch.long, device=input_ids.device)
    attention_mask[:len(title)] = 1
    global_attention_mask = torch.zeros(input_ids.shape, dtype=torch.long, device=input_ids.device)
    global_attention_mask[0] = 1
    return {
      'titles': title,
      'input_ids': input_ids.flatten(),
      'attention_mask': attention_mask.flatten(),
      'global_attention_mask': global_attention_mask.flatten(),
      'targets': torch.tensor(target, dtype=torch.float)
    }

Viene scelta la dimensione di batch massima che la GPU riesca a contenere in memoria.

In [ ]:
BATCH_SIZE = 8

In [ ]:
def create_data_loader(text, targets, tokenizer, max_len, batch_size):
  ds = FinancialDataset(
    titles=text,
    targets=targets,
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=BATCH_SIZE,
    shuffle=True
  )

Random seed per la riproducibilità dei test svolti.

In [ ]:
RANDOM_SEED = 21

Split del dataset in dataset per addestramento e dataset per valutazione.

In [ ]:
df_train, df_test = train_test_split(
  df,
  test_size=0.15,
  random_state=RANDOM_SEED
)

In [ ]:
tokenizer = LongformerTokenizer.from_pretrained(MODEL_PATH, return_tensor="pt", truncation=True)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803 [00:00<?, ?B/s]

In [ ]:
train_data_loader = create_data_loader(df_train['title'].to_numpy(), df_train['target'].to_numpy(), tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
test_data_loader = create_data_loader(df_test['title'].to_numpy(), df_test['target'].to_numpy(), tokenizer, MAX_LEN, BATCH_SIZE)

## Rete neurale per classificazione

Viene definito un encoder che a partire dai token id di ogni entry del dataset genererà l'encoding a 768 dimensioni di ogni parola. L'encoding è Longformer, un modello ad attention lineare in grado di superare il limite di 512 parole imposto dai transformer ad attention quadratica. Utilizza una sliding window (in questo caso di 32 token) per calcolare l'attention locale ed una eventuale global attention, che però non verrà utilizzata in questo caso in quanto troppo onerosa in termini di memoria.

L'output dell'encoder per ogni entry è un tensore 1024x768 (parole x dimensioni). Per utilizzare questo output nella rete neurale di classificazione, viene effettuata una operazione di pooling che riduce l'output a un tensore 1x768.

Il modello Longformer di encoding utilizza i pesi di un precedente addestramento. Non sono stati riscontrati miglioramenti qualora si addestri l'encoder assieme alla rete neurale, dunque esso non viene addestrato per risparmiare memoria evitando di aggiornare i pesi.

In [ ]:
encoder = LongformerModel.from_pretrained(MODEL_PATH, attention_window = 1024, output_hidden_states=True).to(device)

Downloading:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-large-4096 were not used when initializing LongformerModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Il modello di classificazione è invece composto da 3 blocchi che comprendono trasformazione lineare, batch normalization, funzione di attivazione e dropout. Infine è posto un output layer lineare senza funzione di attivazione.

In [ ]:
class Classifier(nn.Module):
  def __init__(self):
        super(Classifier, self).__init__()

        self.block1 = self.lin_block(c_in=encoder.config.hidden_size*4, c_out=256, dropout=0.1)
        self.block2 = self.lin_block(c_in=256, c_out=64, dropout=0.1)
        self.block3 = self.lin_block(c_in=64, c_out=16, dropout=0.1)
        self.out = nn.Linear(16, 1)

  def forward(self, embedding):
        x = self.block1(embedding)
        x = self.block2(x)
        x = self.block3(x)
        return self.out(x)
  
  def lin_block(self, c_in, c_out, dropout,  **kwargs):
        seq_block = nn.Sequential(
            nn.Linear(c_in, c_out),
            nn.BatchNorm1d(num_features=c_out),
            nn.Tanh(),
            nn.Dropout(p=dropout)
        )
        return seq_block

In [ ]:
model = Classifier().to(device)

Viene definito un ottimizzatore e una funzione d'errore. La funzione di errore utilizzata è _binary cross entropy_ in quanto si tratta di un problema di classificazione binaria. Viene utilizzata la versione _with logits_ in quanto gli output della rete neurale non passano per una funzione di attivazione.

In [ ]:
EPOCHS = 6
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.05)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.BCEWithLogitsLoss().to(device)

Viene definita una funzione per costruire gli embedding a partire dall'output dell'encoder: viene concatenato il primo token degli ultimi 4 output, dunque si ha un tensore a 3072 dimensioni.

In [ ]:
def compute_embeddings(model, input_ids, attention_mask, global_attention_mask): 
    with torch.no_grad():
        hidden_states = encoder(input_ids, attention_mask, global_attention_mask)[2]
        pooled_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)
        out = pooled_output[:, 0, :]
        return out
        
    

Secondo le indicazioni della [documentazione PyTorch](https://pytorch.org/docs/stable/optim.html), vengono definiti gli step per l'addestramento e la valutazione del modello. 

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, scheduler, n_examples, device):
  model = model.train()
  losses = []
  correct_predictions = 0
  step = 0
  for d in data_loader:
      step += 1
      optimizer.zero_grad() # clears previous gradients
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      global_attention_mask = d["global_attention_mask"].to(device)
      targets = d["targets"].to(device)
      
      pooled_output = compute_embeddings(model, input_ids, attention_mask, global_attention_mask).to(device)
      outputs = model(pooled_output)
      preds = outputs>0    
      loss = loss_fn(outputs, targets.unsqueeze(1)) # computes loss
      correct_predictions += torch.sum(torch.transpose(preds, 0, 1) == targets)
      losses.append(loss.item())
      loss.backward() 
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
      optimizer.step() # optimizer takes step based on gradients
      scheduler.step() 
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  step = 0
  with torch.no_grad(): # gradient computation disabled for evalutaion
      for d in data_loader:
        step += 1
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        global_attention_mask = d["global_attention_mask"].to(device)
        targets = d["targets"].to(device)
        pooled_output = compute_embeddings(model, input_ids, attention_mask, global_attention_mask).to(device)
        outputs = model(pooled_output)
        preds = (outputs>0)    
        loss = loss_fn(outputs, targets.unsqueeze(1))
        correct_predictions += torch.sum(torch.transpose(preds, 0, 1) == targets)
        losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

Al termine di ogni epoca di addestramento viene effettuato un riscontro sul test set: se l'errore su questo set è migliore rispetto alle epoche precedenti, vengono aggiornati i pesi che verranno salvati come migliori per il modello.

In [ ]:
history = defaultdict(list)
least_loss = 1000
for epoch in range(EPOCHS):
  
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    scheduler,
    len(df_train),
    device
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')
  
  val_acc, val_loss = eval_model(
    model,
    test_data_loader,
    loss_fn,
    device,
    len(df_test)
  )


  print(f'Val   loss {val_loss} accuracy {val_acc}')
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if float(val_loss) < float(least_loss):
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_loss = val_loss

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/6


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Train loss 0.6996469301421467 accuracy 0.5356052899287894
Val   loss 0.7021814449266954 accuracy 0.5360230547550432
Epoch 2/6
Train loss 0.6796797589073337 accuracy 0.5686673448626653
Val   loss 0.7002051594582471 accuracy 0.4841498559077809
Epoch 3/6
Train loss 0.6661913157478581 accuracy 0.5996948118006104
Val   loss 0.6996487826108932 accuracy 0.5273775216138328
Epoch 4/6
Train loss 0.6599369456128377 accuracy 0.6185147507629705
Val   loss 0.6992105977101759 accuracy 0.515850144092219
Epoch 5/6
Train loss 0.6532275405356555 accuracy 0.6495422177009156
Val   loss 0.6991525319489565 accuracy 0.5216138328530259
Epoch 6/6
Train loss 0.6527024287033857 accuracy 0.638351983723296
Val   loss 0.6962559616023843 accuracy 0.515850144092219


## Conclusione

Terminato l'addestramento, vengono caricati i pesi migliori per fare una valutazione finale del modello in cui ogni 5 batch viene stampato l'output

In [ ]:
WEIGHTS = 'best_model_state.bin'
model.load_state_dict(torch.load(WEIGHTS))

<All keys matched successfully>

In [ ]:
def final_model_evaluation(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  step = 0
  with torch.no_grad(): # gradient computation disabled for evalutaion
      for d in data_loader:
        step += 1
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        global_attention_mask = d["global_attention_mask"].to(device)
        targets = d["targets"].to(device)
        pooled_output = compute_embeddings(model, input_ids, attention_mask, global_attention_mask).to(device)
        outputs = model(pooled_output)
        preds = (outputs>0)    
        loss = loss_fn(outputs, targets.unsqueeze(1))
        correct_predictions += torch.sum(torch.transpose(preds, 0, 1) == targets)
        losses.append(loss.item())
        if(step%5==0):
          print("Network output: ", outputs, "predictions: ", torch.transpose(preds, 0, 1), "targets: ", targets)
          print("Step: ", step, ", batch loss: ", loss.item(), ", batch correct preds:", torch.sum(torch.transpose(preds, 0, 1) == targets))
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
  val_acc, val_loss = final_model_evaluation(
    model,
    test_data_loader,
    loss_fn,
    device,
    len(df_test)
  ) 

  print(f'Final model: loss {val_loss} accuracy {val_acc}')

Network output:  tensor([[0.4392],
        [0.6640],
        [0.1776],
        [0.1452],
        [0.5781],
        [0.0717],
        [0.1275],
        [0.4420]], device='cuda:0') predictions:  tensor([[True, True, True, True, True, True, True, True]], device='cuda:0') targets:  tensor([0., 0., 1., 1., 1., 1., 0., 0.], device='cuda:0')
Step:  5 , batch loss:  0.7559908032417297 , batch correct preds: tensor(4, device='cuda:0')
Network output:  tensor([[ 0.0756],
        [-0.4417],
        [ 0.2229],
        [ 0.2564],
        [ 0.1386],
        [ 0.1792],
        [-0.0223],
        [-0.0334]], device='cuda:0') predictions:  tensor([[ True, False,  True,  True,  True,  True, False, False]],
       device='cuda:0') targets:  tensor([0., 0., 1., 0., 1., 0., 0., 1.], device='cuda:0')
Step:  10 , batch loss:  0.6813291311264038 , batch correct preds: tensor(4, device='cuda:0')
Network output:  tensor([[ 0.0724],
        [-0.2002],
        [ 0.2630],
        [ 0.2192],
        [-0.0368],
    

Il modello con i pesi migliori ha una accuracy poco maggiore del 51%: questo risultato può essere considerato positivo ed applicabile al mercato finanziario, ma è comunque inferiore a quanto mostrato nel paper sopracitato.